This notebook is to load predicted mask for hard-disk and calculate Segmentation evaluation metics.
predicted mask is the output of ScaleNASv2 Test and save predicted mask.ipynb

!cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_test.py .
!cp -v /mnt/fast-data/mjc/AutoRECIST/Codes/ScaleNAS/ScaleNASv1/tools/utils_metrics_3d.py .

In [11]:
# coding: utf-8

# # This file is for segmetation metrics evaluation in 3D
# Edited by Jingchen around 06/20/2021
# This file is after ScaleNAS test which save predition into png images.
# This file load png images as predicted contours in 2D
# load cache pkl as gold-standard contours in 2D
# The stack 2D based on dicom-header to get 3D
# Evaluate 3D metics of dice, IoU, over-segmetation, and under-segmetation
# 
%reload_ext autoreload
%autoreload 2

import os
import logging
import numpy as np
import _init_paths

from config import cfg
from config import update_config
from utils_test import *

import cv2
from PIL import Image
import torch.nn.functional as F
from utils.utils import get_confusion_matrix
def convert_name(name):
    new = name.replace('/','_')
    return new

def get_palette(n):
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette
    
# def save_pred(preds, sv_path, name):

#     preds = preds.cpu().numpy().copy()
#     preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
#     for i in range(preds.shape[0]):
#         cv2.imwrite(os.path.join(sv_path, convert_name(name[i])) , preds[i])

        
def save_pred( preds, sv_path, name):
    palette = get_palette(256)
    preds = preds.cpu().numpy().copy()
    preds = np.asarray(np.argmax(preds, axis=1), dtype=np.uint8)
    for i in range(preds.shape[0]):
        pred = preds[i]
        save_img = Image.fromarray(pred)
        save_img.putpalette(palette)
        save_img.save(os.path.join(sv_path, convert_name(name[i]) ))
        

def testval_lesion(config, test_dataset, testloader, model,
            sv_dir='', sv_pred=True, device = None):
    model.eval()
    confusion_matrix = np.zeros(
        (config.DATASET.NUM_CLASSES, config.DATASET.NUM_CLASSES))
    with torch.no_grad():
        for index, batch in enumerate(testloader):
            image, label, _, batchname = batch
            size = label.size()
            if device is None:
                image = image.cuda()
                label = label.long().cuda()
            else:
                image = image.to(device)
                label = label.long().to(device)

            pred = model(image)
            if pred.size()[-2] != size[-2] or pred.size()[-1] != size[-1]:
                pred = F.upsample(pred, (size[-2], size[-1]),
                                  mode='bilinear')

            confusion_matrix += get_confusion_matrix(
                label,
                pred,
                size,
                config.DATASET.NUM_CLASSES,
                config.TRAIN.IGNORE_LABEL)

            if sv_pred:
                sv_path = os.path.join(sv_dir, 'test_val_results')
                if not os.path.exists(sv_path):
                    os.mkdir(sv_path)
                save_pred(pred, sv_path, batchname)

            if index % 100 == 0:
                logging.info('processing: %d images' % index)
                pos = confusion_matrix.sum(1)
                res = confusion_matrix.sum(0)
                tp = np.diag(confusion_matrix)
                IoU_array = (tp / np.maximum(1.0, pos + res - tp))
                mean_IoU = IoU_array.mean()
                logging.info('mIoU: %.4f' % (mean_IoU))

    pos = confusion_matrix.sum(1)
    res = confusion_matrix.sum(0)
    tp = np.diag(confusion_matrix)
    pixel_acc = tp.sum() / pos.sum()
    mean_acc = (tp / np.maximum(1.0, pos)).mean()
    IoU_array = (tp / np.maximum(1.0, pos + res - tp))
    mean_IoU = IoU_array.mean()

    return mean_IoU, IoU_array, pixel_acc, mean_acc

In [12]:
cache_path = './cache/'
name = 'inference'

DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AUTO_RECIST CIA-LAB Image and Contour 2020-10-01.xlsx'
# json_name = 'CUIMC20201027.json'

# DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AMGEN_20020408_CIA-LAB_Image_And_Contour_2020-10-21.xlsx'
# json_name = 'AMGEN_20020408_20201027.json'

# DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AMGEN_PRIME_CIA-LAB_Image_And_Contour_2020-10-21.xlsx'
# json_name = 'AMGEN_PRIME_20201027.json'

cache_filepath = os.path.join(cache_path, name+'_gt_roidb.pkl')
print('Loading cached gt_roidb from %s', cache_filepath)
with open(cache_filepath, 'rb') as fp:
    cached_roidb = pickle.load(fp)
    
roidb = cached_roidb


# In[25]:

from utils_test import __get_annotation__

sv_dir = mask_name = 'mask_1988'
sv_path = os.path.join(sv_dir, 'test_val_results')

all_boxes = [ [ np.zeros((0,5),dtype="float32") for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]
all_segms = [ [ [] for _ in range(len(roidb)) ] for _ in range( cfg.DATASET.NUM_CLASSES) ]

for i in range(len(roidb)):
    if i%500==0:
        print('='*40,i)

    one = roidb[i]
    onename = one['image']
    pred_im = Image.open(os.path.join(sv_path, convert_name(onename) ))
    pred = np.array(pred_im)
    for j in range(cfg.DATASET.NUM_CLASSES):
        mask = np.asarray( pred==j , dtype=np.uint8)
        if np.sum(mask > 0) <= 3 :
            continue
        segmentation, bbox, area = __get_annotation__(mask , xywh = False , bbox_score=True)
        if segmentation and bbox:
            all_segms[j][i] = segmentation
            all_boxes[j][i] = bbox

Loading cached gt_roidb from %s ./cache/inference_gt_roidb.pkl
======================================== 0
======================================== 500
======================================== 1000
======================================== 1500
======================================== 2000
======================================== 2500
======================================== 3000
======================================== 3500
======================================== 4000
======================================== 4500
======================================== 5000
======================================== 5500
======================================== 6000
======================================== 6500
======================================== 7000
======================================== 7500
======================================== 8000
======================================== 8500
======================================== 9000
======================================== 9500
=====================

In [13]:
site_ix = 8
gt_boxes = get_gt_boxes(cached_roidb,site_ix)
avgFP=[0.5,1,2,3,4,8,16,32,64]
iou_th=0.5

stack_box = all_boxes[site_ix]
stack_box = [np.array(b,dtype=np.float32) for b in stack_box]
result, valid_avgFP = sens_at_FP(stack_box, gt_boxes, avgFP, iou_th)
print('='*40)
for recall,fp in zip(result,valid_avgFP):
    print('Recall@%.1f=%.2f%%' % (fp, recall*100))
#TODO: when num of valid_avgFP < 6,is FROC correct?
print('Mean FROC is %.2f'% np.mean(np.array(result[:6])*100))
print('='*40)


sens, fp_per_img, scores_th = FROC(stack_box, gt_boxes, iou_th)

/mnt/fast-disk1/mjc/AutoRecist/Codes/ScaleNAS_9Slices/tools/utils_test.py:225: RuntimeWarning: invalid value encountered in true_divide
  sens = np.array(tps, dtype=float) / nGt


FROC:FP in no-lesion-images:  5913
Recall@0.1=nan%
Mean FROC is nan
FROC:FP in no-lesion-images:  5913


In [23]:
site_list = [site_ix] #8 is liver
D_CT = {} #contain 1:roidb(gt from cache by MVPNet)
        #2:predicted bboxes
        #3:predicted segmentations
for i , aroidb in enumerate(roidb):
    dicom_path , png_name = os.path.split(aroidb['image'])
    slice_no , _= os.path.splitext(png_name)
    slice_no = int(slice_no)
    if slice_no != aroidb['slice_no']:
        print(dicom_path,slice_no,aroidb['slice_no'])

    segmentations = {}
    bboxes = {}
    for j in site_list:
        segmentations[j] = all_segms[j][i]
        bboxes[j] = all_boxes[j][i]

    if dicom_path not in D_CT:
        D_CT[dicom_path] = {}
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]
    else:
        D_CT[dicom_path][slice_no] = [aroidb , bboxes , segmentations]


# In[37]:

import pandas as pd
DL_info = '/mnt/fast-data/mjc/AutoRECIST/Inputs/PDS_AUTO_RECIST CIA-LAB Image and Contour 2020-10-01.xlsx'
df = pd.read_excel(DL_info)

def pd_str_replace(df , col, ori, new):
    if isinstance(col , str):
        df[col] = df[col].str.replace(ori,new, case = False) 
    elif isinstance(col, list):
        for one in col:
            pd_str_replace(df , one, ori, new)
    else:
        raise('col instence should be str or list')

pd_str_replace(df, ['Image File Path' , 'Contour File Path'], "X:" , "/mnt/W-drive")
pd_str_replace(df, ['Image File Path' , 'Contour File Path'], r"\\" , "/")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrialDone/FNIH_VOLPACK", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Image File Path'], "/mnt/W-drive/ClinicalTrials", "/mnt/fast-disk1/mjc/AutoRecist/Inputs")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw/PDS_AUTO_RECIST", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToRaw", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")
pd_str_replace(df, ['Contour File Path'], "/mnt/W-drive/ConvWeasisToMatlab", "/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw")


# In[52]:

sys.path.append('/mnt/fast-data/mjc/DeepLesion/Codes/CollectData/')

from print_weasis_raw_data import WeasisRawDataPrinter
from read_weasis_raw_data import WeasisRawFileReader


import os
import numpy as np
import pydicom
import matplotlib.pylab as plt


data_path = ""
metadata_path = "/mnt/fast-data/mjc/AutoRECIST/Outputs/Dicom_header"
if not os.path.exists(metadata_path):
    os.mkdir(metadata_path)


D_dir2header_df = {}


def get_dicom_header_df(image_dir , labels = []):
    global D_dir2header_df
    if image_dir in D_dir2header_df:
        return D_dir2header_df[image_dir]

    # image_dir = row['Image File Path']


    labels = ['ImageName','InstanceNumber',
            'BitsAllocated', 'BitsStored', 'Columns', 'HighBit', 
            'ImageOrientationPatient_0', 'ImageOrientationPatient_1', 'ImageOrientationPatient_2',
            'ImageOrientationPatient_3', 'ImageOrientationPatient_4', 'ImageOrientationPatient_5',
            'ImagePositionPatient_0', 'ImagePositionPatient_1', 'ImagePositionPatient_2',
            'Modality', 'PatientID', 'PhotometricInterpretation', 'PixelRepresentation',
            'PixelSpacing_0', 'PixelSpacing_1', 'RescaleIntercept', 'RescaleSlope', 'Rows', 'SOPInstanceUID',
            'SamplesPerPixel', 'SeriesInstanceUID', 'StudyID', 'StudyInstanceUID', 
            'WindowCenter', 'WindowWidth', 
        ] if not labels else labels

    data = {l: [] for l in labels}
    
    ctList = os.listdir(image_dir)
    ctList.sort()

    for image in ctList:
        if '.dcm' not in image:
            continue
        if os.path.getsize(os.path.join(image_dir, image)) < 5*1024:
            print('%s size < 5kb skiped!'%os.path.join(image_dir, image) )
            continue
        data["ImageName"].append(image)

        ds = pydicom.dcmread(os.path.join(image_dir, image))
        for metadata in ds.dir():
            if metadata not in data and metadata not in ['ImageOrientationPatient','ImagePositionPatient','PixelSpacing']:
                continue
            if metadata != "PixelData":
                metadata_values = getattr(ds, metadata)
                if type(metadata_values) == pydicom.multival.MultiValue and metadata not in ["WindowCenter", "WindowWidth"]:
                    for i, v in enumerate(metadata_values):
                        data[f"{metadata}_{i}"].append(v)  
                else:
                    if type(metadata_values) == pydicom.multival.MultiValue and metadata in ["WindowCenter", "WindowWidth"]:
                        data[metadata].append(metadata_values[0])
                    else:
                        data[metadata].append(metadata_values)

    df_image = pd.DataFrame(data).set_index("InstanceNumber")
    D_dir2header_df[image_dir] = df_image
    return df_image

In [29]:
def reduceFP(vol):
    

[8]

In [51]:
# In[53]:

from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():

    if row.Location != ix2labelname(site_ix):
#         only calculate liver
        continue
    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT')

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1)
    Metrics_vol.extend(hit)
    
    if _%100==0 or _<10 or _ in [20,50]:
        print(_)
        df_metrics = pd.DataFrame(Metrics_vol, 
                                  columns = ['file_name','igt','merge','#gt','#pred',
                                             'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                             'area_gt','area_pred','intersection','union']) 
        df_metrics.to_csv('temp_%s_.csv'%'ScaleNAS_Q5_9Slices_CUIMC')

0
1
2
3
4
5
6
7
8
9
100
200
300
400
500
600
700
800
900
1000
1100


In [60]:
D_CT.keys()

dict_keys(['/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0001/D2018_12_03/E2952/CT/S0002_6566', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0003/D2019_10_07/E2122/CT/S0002_2131', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0004/D2019_03_04/E9409/CT/S0016_5317', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0005/D2019_01_03/E7914/CT/S0004_6693', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0006/D2018_12_11/E8314/CT/S0002_5268', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0007/D2019_02_26/E8395/CT/S0004_4916', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0008/D2019_12_19/E2450/CT/S0002_7253', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0009/D2020_01_22/E0578/CT/S0018_3927', '/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0010/D2019_01_04/E0016/CT/S0003_7994', '/mnt/fas

In [104]:
print(name)
pd.options.display.float_format = "{:.3f}".format
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

inference


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,375.000,375.000,375.000,375.000,375.000,375.000,375.000,375.000,375.000,375.000,375.000,375.000
mean,1.005,0.000,1.000,10.373,0.296,0.405,9.754,0.327,8704.096,27991.131,6998.936,29696.291
std,0.073,0.000,0.000,8.166,0.236,0.289,31.803,0.347,32418.059,75275.477,27737.905,78145.998
min,1.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,17.000,0.000,0.000,17.000
5%,1.000,0.000,1.000,2.000,0.000,0.000,0.000,0.002,78.400,0.000,0.000,125.700
25%,1.000,0.000,1.000,4.000,0.042,0.081,0.212,0.045,274.000,291.500,118.500,620.000
50%,1.000,0.000,1.000,8.000,0.294,0.454,0.749,0.176,757.000,1339.000,447.000,2028.000
75%,1.000,0.000,1.000,17.000,0.496,0.663,2.787,0.513,3177.500,13191.000,2103.500,15949.000
95%,1.000,0.000,1.000,29.000,0.661,0.796,49.282,1.000,32485.700,152528.600,25552.300,185572.400
max,2.000,0.000,1.000,29.000,0.884,0.938,257.271,1.000,274688.000,423728.000,242791.000,457032.000


In [106]:
df_metrics[df_metrics.dice_score>0.25].describe([.5])

,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,241.000,241.000,241.000,241.000,241.000,241.000,241.000,241.000,241.000,241.000,241.000,241.000
mean,1.008,0.000,1.000,10.062,0.442,0.596,1.078,0.196,11932.734,16622.801,9982.668,18572.867
std,0.091,0.000,0.000,8.038,0.160,0.157,1.079,0.196,39784.745,52306.939,34082.967,57904.737
min,1.000,0.000,1.000,1.000,0.143,0.251,0.000,0.000,47.000,36.000,20.000,83.000
50%,1.000,0.000,1.000,8.000,0.445,0.616,0.706,0.128,806.000,1228.000,601.000,1419.000
max,2.000,0.000,1.000,29.000,0.884,0.938,5.958,0.855,274688.000,423728.000,242791.000,457032.000


In [105]:
dfinnermerge = pd.merge(df_metrics,df,how='inner',left_on='file_name' , right_on='Contour File Path')

for col in dfinnermerge.columns.tolist():
    print(col , len(set(dfinnermerge[col].tolist() )) )



pts = dfinnermerge["Image File Path"].values.tolist()
FPs = []
for onept in list(set(pts)):
    df_onept = dfinnermerge[dfinnermerge["Image File Path"]==onept]
    assert( min(df_onept["#pred"]) == max(df_onept["#pred"]))
    fp = max(df_onept["#pred"])
    FPs.append(fp)
print("="*80)
print( "In total, {} CT series; {} AI detections ".format( len(FPs) , sum(FPs) ) )

dices = dfinnermerge.dice_score.tolist()

for th in [0, 0.1, 0.2, 0.25, 0.5]:
    TP = [p>th for p in dices ]
    assert( len(TP) == len(dices))
    fprate = ( sum(FPs) - sum(TP) ) / len(FPs)
    print(f"sensitivity is {sum(TP)/len(dices):.3f}({sum(TP)}/{len(dices)}) FP-rate is {fprate:.1f} per CT-serie at threshold {th}")

file_name 375
igt 2
merge 1
#gt 1
#pred 18
iou_score 324
dice_score 324
over_seg 324
under_seg 310
area_gt 341
area_pred 249
intersection 302
union 355
PatientID 90
Image File Path 92
Contour File Path 375
Location 1
Uni 366
Perp 358
Bi 374
Volume 374
In total, 92 CT series; 477 AI detections 
sensitivity is 0.867(325/375) FP-rate is 1.7 per CT-serie at threshold 0
sensitivity is 0.733(275/375) FP-rate is 2.2 per CT-serie at threshold 0.1
sensitivity is 0.677(254/375) FP-rate is 2.4 per CT-serie at threshold 0.2
sensitivity is 0.643(241/375) FP-rate is 2.6 per CT-serie at threshold 0.25
sensitivity is 0.467(175/375) FP-rate is 3.3 per CT-serie at threshold 0.5


In [26]:
print(name)
df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.describe([.05, .25, .5, .75, .95])

inference


,igt,merge,#gt,#pred,iou_score,dice_score,over_seg,under_seg,area_gt,area_pred,intersection,union
count,375.000000,375.0,375.0,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000,375.000000
mean,1.005333,0.0,1.0,15.986667,0.304834,0.417955,9.760835,0.315275,8704.096000,27992.746667,7000.202667,29696.640000
std,0.072932,0.0,0.0,10.538056,0.231398,0.281628,31.801449,0.334399,32418.058594,75274.874767,27737.585294,78145.865876
min,1.000000,0.0,1.0,2.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000,29.000000
5%,1.000000,0.0,1.0,4.000000,0.000000,0.000000,0.000000,0.002282,78.400000,0.000000,0.000000,130.200000
25%,1.000000,0.0,1.0,7.000000,0.069696,0.130308,0.232709,0.045489,274.000000,291.500000,118.500000,620.000000
50%,1.000000,0.0,1.0,13.000000,0.299382,0.460807,0.761290,0.167742,757.000000,1339.000000,447.000000,2028.000000
75%,1.000000,0.0,1.0,27.000000,0.498106,0.664981,2.787067,0.496665,3177.500000,13191.000000,2103.500000,15949.000000
95%,1.000000,0.0,1.0,37.000000,0.660662,0.795648,49.281664,1.000000,32485.700000,152528.600000,25552.300000,185572.400000
max,2.000000,0.0,1.0,37.000000,0.883871,0.938356,257.270806,1.000000,274688.000000,423728.000000,242791.000000,457032.000000


In [53]:
df_metrics.to_csv('%s_liver_seg_metrics_vol_20220106_reduceFP.csv'%'ScaleNAS_Q5_9Slices_CUIMC')

In [28]:
!ls -lht

total 32M
-rw-r--r--. 1 jm4669 domain users 102K Jan  7 12:28 ScaleNAS_Q5_9Slices_CUIMC_liver_seg_metrics_vol_20220106.csv
-rw-r--r--. 1 jm4669 domain users  20K Jan  7 12:27 ScaleNAS_Q5_9Slice Test and save predicted mask.ipynb
-rw-r--r--. 1 jm4669 domain users 130K Jan  7 02:37 ScaleNAS_Q5_9Slices Predicted mask to Metrics CUIMC.ipynb
drwxr-xr-x. 2 jm4669 domain users   59 Jan  7 00:33 cache
-rw-r--r--. 1 jm4669 domain users  52K Jan  6 23:18 ScaleNAS_Q5_9Slices Predicted mask to Metrics Whole CT Allsites.ipynb
-rw-r--r--. 1 jm4669 domain users  12K Jan  6 15:45 utils_metrics_3d.py
drwxr-xr-x. 2 jm4669 domain users  128 Jan  6 15:31 __pycache__
drwxr-xr-x. 3 jm4669 domain users   38 Jan  5 18:38 mask_1988
drwxr-xr-x. 3 jm4669 domain users   28 Jan  5 17:56 log
drwxr-xr-x. 3 jm4669 domain users   20 Jan  5 17:56 output
-rw-r--r--. 1 jm4669 domain users 5.1K Dec 23 15:16 evo_server.py
-rw-r--r--. 1 jm4669 domain users  71K Dec 21 15:06 Search Find a png from json.ipynb
-rw-r--r--. 1 jm

In [ ]:
stop

In [18]:
row['Image File Path']

'/mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO_RECIST/METNET0011/D2019_01_08/E0715/CT/S0003_9046'

In [17]:
row['Contour File Path']

'/mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeasisToRaw/PDS_AUTO_RECIST_RAW/pg2343_METNET0040_2018-12-25_S0004_T0036_1.2.840.113619.2.340.3.1930047367.168.1545407615.81.3153920.raw'

In [22]:
df.iloc[66]

PatientID                                                   METNET0011
Image File Path      /mnt/fast-disk1/mjc/AutoRecist/Inputs/PDS_AUTO...
Contour File Path    /mnt/fast-disk1/mjc/AutoRecist/Inputs/ConvWeas...
Location                                                         liver
Uni                                                            12.5484
Perp                                                           10.1467
Bi                                                             127.325
Volume                                                         684.688
Name: 66, dtype: object

In [20]:
df_image

,ImageName,BitsAllocated,BitsStored,Columns,HighBit,ImageOrientationPatient_0,ImageOrientationPatient_1,ImageOrientationPatient_2,ImageOrientationPatient_3,ImageOrientationPatient_4,...,RescaleIntercept,RescaleSlope,Rows,SOPInstanceUID,SamplesPerPixel,SeriesInstanceUID,StudyID,StudyInstanceUID,WindowCenter,WindowWidth
InstanceNumber,,,,,,,,,,,,,,,,,,,,,
1,I0001.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
2,I0002.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
3,I0003.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
4,I0004.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
5,I0005.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
6,I0006.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
7,I0007.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
8,I0008.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0
9,I0009.dcm,16,16,512,15,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,512,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,1,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,,1.2.392.200036.9116.2.6.1.48.1221398285.154691...,40.0,400.0


In [ ]:
# In[55]:

print(name)
df_miss = pd.DataFrame(Missed_vol, 
                          columns = ['file_name','igt','merge','#gt','#pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_miss.describe([.05, .25, .5, .75, .95])

In [83]:
# In[ ]:

SHOW_LABEL = True
SHOW_BOX = False
SHOW_MASK = True
SHOW_UNION_MASK= False
SHOW_MASK_LABEL = True
SHOW_GT_MASK = False


from utils_metrics_3d import *
keys = list(D_CT.keys())

for k in keys[:5]:
    print(k)
    
    oneCT = remove_single_slice_segms(D_CT[k])
    savepath = '/mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/%s/CTs_%s/%s'%(mask_name,name,convert_name_compact(k))
    if not os.path.exists(savepath):
        os.makedirs(savepath)
    
    for s in oneCT:
        aroidb , bboxes , segmentations = oneCT[s]

        image_path = os.path.join(aroidb['image'])
    
#         CT_windowing = get_proper_CT_windowing()
#         if CT_windowing:
#             HU1, HU2 = CT_windowing
#         else:
#             [HU1, HU2 ] = aroidb['windows']
            

        image = load_image(image_path, HU1, HU2)
        height,width = image.shape
    #     plt.imshow(image)
        image = np.dstack((image,image,image))
        
        if SHOW_MASK:
            for j in site_list:
                contours = segmentations[j]
                colors = compute_colors_for_labels(j)
                label = ix2labelname(j)
                for c in contours:
                    c = np.reshape(c,(-1,2))
                    if c.shape[0]:
                        image = cv2.drawContours(image, [np.int64( c  )], -1, colors, 1)
                        if SHOW_MASK_LABEL:
                            x,y,_,_ =ploy2boxes(c)
                            template = "{}"
                            if len(label)>=9:
                                label=label[:3]+label[-3:]
                
                            s = template.format(label)
                            cv2.putText(
                                image, s, (x, y), cv2.FONT_HERSHEY_SIMPLEX, .5, colors, 1
                            )  
        

        cv2.imwrite( os.path.join(savepath, convert_name_compact( aroidb['image'] ) ) , image )

/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0001/D2018_12_03/E2952/CT/S0002_6566
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0003/D2019_10_07/E2122/CT/S0002_2131
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0004/D2019_03_04/E9409/CT/S0016_5317
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0005/D2019_01_03/E7914/CT/S0004_6693


/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0001/D2018_12_03/E2952/CT/S0002_6566


In [88]:
for i in range(1,22):
    print(i , compute_colors_for_labels(i) , ix2labelname(i))

1 [1, 127, 31] abdomen
2 [2, 254, 62] abdomen LN
3 [3, 126, 93] adrenal
4 [4, 253, 124] axillary LN
5 [5, 125, 155] bone
6 [6, 252, 186] inguinal LN
7 [7, 124, 217] kidney
8 [8, 251, 248] liver
9 [9, 123, 24] lung
10 [10, 250, 55] mediastinum LN
11 [11, 122, 86] neck LN
12 [12, 249, 117] ovary
13 [13, 121, 148] pancreas
14 [14, 248, 179] pelvic LN
15 [15, 120, 210] pelvis
16 [16, 247, 241] pleural
17 [17, 119, 17] retroperitoneal LN
18 [18, 246, 48] soft tissue
19 [19, 118, 79] spleen
20 [20, 245, 110] stomach
21 [21, 117, 141] thyroid


In [ ]:

all_segms[8][5]


# In[ ]:

all_boxes[8][5]


# In[ ]:




# In[ ]:
 hit, miss = vols_seg_results(mask_vol , vol_pred, CTname=ss, gt_keep_largest=1)

In [ ]:
connectivity = 2
from skimage import measure,color
labels_gt=measure.label(mask_vol,connectivity=connectivity)
l_gt,c_gt = np.unique(labels_gt , return_counts=True)


In [31]:
a = np.zeros((3,4,2))
ix = np.where(a==0)

In [45]:
x,y,z = np.where(a==0)
x

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2])

In [47]:

ix = x>1

In [103]:

#-----------------------------------------------------------------------
class Color:
    """
    A Color object models an Color color.
    """

    def __init__(self, r=0, g=0, b=0):
        """
        Construct self such that it has the given red (r),
        green (g), and blue (b) components.
        """
        self._r = r  # Red component
        self._g = g  # Green component
        self._b = b  # Blue component

    def getRed(self):
        """
        Return the red component of self.
        """
        return self._r

    def getGreen(self):
        """
        Return the green component of self.
        """
        return self._g

    def getBlue(self):
        """
        Return the blue component of self.
        """
        return self._b

    def __str__(self):
        """
        Return the string equivalent of self, that is, a
        string of the form '(r, g, b)'.
        """
        #return '#%02x%02x%02x' % (self._r, self._g, self._b)
        return '[' + str(self._r) + ', ' + str(self._g) + ', ' + \
            str(self._b) + ']'

#-----------------------------------------------------------------------

"""
The lesion color table defines some popular Color objects for lesions.
"""
class LesionColorTable:
    #-------------------------------RED-------------------------------
    VERY_DARK_RED = Color(0x80, 0x00, 0x00)

    DARK_RED = Color(0xc0, 0x00, 0x00)

    LIGHT_RED = Color(0xFF, 0x40, 0x40)

    VERY_LIGHT_RED = Color(0xFF, 0x80, 0x80)
    
    #-------------------------------BLUE-------------------------------
    VERY_DARK_BLUE = Color(0x00, 0x00, 0x80)

    DARK_BLUE = Color(0x00, 0x00, 0xC0)

    LIGHT_BLUE = Color(0x40, 0x40, 0xFF)

    VERY_LIGHT_BLUE = Color(0x80, 0x80, 0xFF)
    
    #-------------------------------GREEN-------------------------------
    VERY_DARK_GREEN = Color(0x00, 0x80, 0x00)

    DARK_GREEN = Color(0x00, 0xC0, 0x00)

    LIGHT_GREEN = Color(0x40, 0xFF, 0x40)

    VERY_LIGHT_GREEN = Color(0x80, 0xFF, 0x80)
    
    #-------------------------------YELLOW-------------------------------
    VERY_DARK_YELLOW = Color(0x80, 0x80, 0x00)

    DARK_YELLOW = Color(0xC0, 0xC0, 0x00)

    LIGHT_YELLOW = Color(0xFF, 0xFF, 0x40)

    VERY_LIGHT_YELLOW = Color(0xFF, 0xFF, 0x80)
    
    #-------------------------------MAGENTA-------------------------------
    VERY_DARK_MAGENTA = Color(0x80, 0x00, 0x80)

    DARK_MAGENTA = Color(0xC0, 0x00, 0xC0)

    LIGHT_MAGENTA = Color(0xFF, 0x40, 0xFF)

    VERY_LIGHT_MAGENTA = Color(0xFF, 0x80, 0xFF)

    #-------------------------------CYAN-------------------------------
    VERY_DARK_CYAN = Color(0x00, 0x80, 0x80)

    DARK_CYAN = Color(0x00, 0xC0, 0xC0)

    LIGHT_CYAN = Color(0x40, 0xFF, 0xFF)

    VERY_LIGHT_CYAN = Color(0x80, 0xFF, 0xFF)

    #-------------------------------COLOR_TABLE-------------------------------
    COLOR_TABLE = [
        # Light
        LIGHT_RED,
        LIGHT_BLUE,
        LIGHT_GREEN,
        LIGHT_YELLOW, 
        LIGHT_MAGENTA, 
        LIGHT_CYAN,

        # Very Light
        VERY_LIGHT_RED, 
        VERY_LIGHT_BLUE,
        VERY_LIGHT_GREEN,
        VERY_LIGHT_YELLOW, 
        VERY_LIGHT_MAGENTA, 
        VERY_LIGHT_CYAN,

        # Dark
        DARK_RED, 
        DARK_BLUE,
        DARK_GREEN,
        DARK_YELLOW, 
        DARK_MAGENTA, 
        DARK_CYAN,

        # Very dark
        VERY_DARK_RED,
        VERY_DARK_BLUE,
        VERY_DARK_GREEN,
        VERY_DARK_YELLOW,
        VERY_DARK_MAGENTA,
        VERY_DARK_CYAN
    ]

    def getColor(tumorIndex):
        if (tumorIndex >= 0):
            return LesionColorTable.COLOR_TABLE[tumorIndex % len(LesionColorTable.COLOR_TABLE)]


In [102]:
for i in range(0,20,5):
    lesion_color = LesionColorTable.getColor(i)
    print(i,lesion_color, compute_colors_for_labels(i) )

0 [64, 64, 255] [0, 0, 0]
5 [128, 128, 255] [5, 125, 155]
10 [0, 0, 192] [10, 250, 55]
15 [0, 0, 128] [15, 120, 210]


In [100]:
for i in range(1,24):
    lesion_color = LesionColorTable.getColor(i)
    print(i,lesion_color, compute_colors_for_labels(i) , ix2labelname(i))

1 [64, 255, 64] [1, 127, 31] abdomen
2 [255, 255, 64] [2, 254, 62] abdomen LN
3 [255, 64, 255] [3, 126, 93] adrenal
4 [64, 255, 255] [4, 253, 124] axillary LN
5 [128, 128, 255] [5, 125, 155] bone
6 [128, 255, 128] [6, 252, 186] inguinal LN
7 [255, 255, 128] [7, 124, 217] kidney
8 [255, 128, 255] [8, 251, 248] liver
9 [128, 255, 255] [9, 123, 24] lung
10 [0, 0, 192] [10, 250, 55] mediastinum LN
11 [0, 192, 0] [11, 122, 86] neck LN
12 [192, 192, 0] [12, 249, 117] ovary
13 [192, 0, 192] [13, 121, 148] pancreas
14 [0, 192, 192] [14, 248, 179] pelvic LN
15 [0, 0, 128] [15, 120, 210] pelvis
16 [0, 128, 0] [16, 247, 241] pleural
17 [128, 128, 0] [17, 119, 17] retroperitoneal LN
18 [128, 0, 128] [18, 246, 48] soft tissue
19 [0, 128, 128] [19, 118, 79] spleen
20 [64, 64, 255] [20, 245, 110] stomach
21 [64, 255, 64] [21, 117, 141] thyroid


KeyError: 22

In [107]:
from utils_metrics_3d import *
Metrics_vol = []
Missed_vol = []
previous_CT_ss = ''
for _,row in df.iterrows():

    if row.Location != ix2labelname(site_ix):
#         only calculate liver
        continue
    df_image = get_dicom_header_df( row['Image File Path'] )
    instanceNumber_list = df_image.index.to_list()
    D_z_index = instanceNumber2Matrix_z_index(instanceNumber_list)

    reader = WeasisRawFileReader()
    weasis_raw_data = reader.read_weasis_raw_file(row['Contour File Path'])

    ss = row['Image File Path']
    ss = ss.replace('/Inputs/' , '/Pngs/')
    if ss not in D_CT:
        print(ss , 'not in D_CT')

    mask_vol = initialize_mask_vol(weasis_raw_data , D_z_index)

    slice_list = weasis_raw_data.get_instance_number_array()
    for j, one in enumerate(slice_list):
        file_name = InstanceNumber2file_name(df_image, one)
        file_name = os.path.join( row['Image File Path'] , file_name)   
        mask = weasis_raw_data.get_mask_image_2d(j)
        mask_vol[D_z_index[one]] = mask

    if ss != previous_CT_ss:
        oneCT = D_CT[ss]
        vol_gt , vol_pred = get_gt_and_pred_vols( oneCT, site_list, mask_vol.shape, D_z_index, union_mask=False )
        previous_CT_ss = ss
    hit = vols_seg_results(mask_vol , vol_pred, CTname=row['Contour File Path'], gt_keep_largest=1 , pred_keep_best=False)
    Metrics_vol.extend(hit)
    
    if _%100==0 or _<10 or _ in [20,50]:
        print(_)
        df_metrics = pd.DataFrame(Metrics_vol, 
                                  columns = ['file_name','i_gt','i_pred','#gt','#pred',
                                             'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                             'area_gt','area_pred','intersection','union']) 
        df_metrics.to_csv('all_TPsFPs_%s_.csv'%'ScaleNAS_Q5_9Slices_CUIMC')

df_metrics = pd.DataFrame(Metrics_vol, 
                          columns = ['Lesion_file_name','i_gt','i_pred','total_gt','total_pred',
                                     'iou_score', 'dice_score', 'over_seg' , 'under_seg',
                                     'area_gt','area_pred','intersection','union']) 
df_metrics.to_csv('all_TPsFPs_%s_.csv'%'ScaleNAS_Q5_9Slices_CUIMC')

0
7
9
50
100
200
400
700
800
900
1000


In [114]:
name="CUIMC_9Slices"
%run -i Save_Image_Masks.py

site_list to save is  [8]
Save path is /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_9Slices_liver/
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0001/D2018_12_03/E2952/CT/S0002_6566
Image masks save to: /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_9Slices_liver/PDS_AUTO_RECIST_METNET0001_D2018_12_03_E2952_CT_S0002_6566
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0002/D2019_01_02/E1326/CT/S0007_3337
Image masks save to: /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_9Slices_liver/PDS_AUTO_RECIST_METNET0002_D2019_01_02_E1326_CT_S0007_3337
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0003/D2019_10_07/E2122/CT/S0002_2131
Image masks save to: /mnt/fast-disk1/mjc/AutoRecist/Outputs/ScaleNAS_Q5_9Slices/mask_1988/CTs_CUIMC_9Slices_liver/PDS_AUTO_RECIST_METNET0003_D2019_10_07_E2122_CT_S0002_2131
/mnt/fast-disk1/mjc/AutoRecist/Pngs/PDS_AUTO_RECIST/METNET0004/

In [116]:
site_list

[8]

In [112]:
mask_name

'mask_1988'